# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものです。

## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [ ]:
import pandas as pd
import numpy as np

## データの読み込み

In [ ]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[((df_receipt["amount"] >= 1000) | (df_receipt["quantity"] >= 5)) & (df_receipt["customer_id"] == "CS018205000001")].loc[:, ["sales_ymd","customer_id","product_cd","quantity","amount"]]

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [ ]:
df_customer.sort_values(by="birth_day").head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

104681

## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
len(df_receipt["customer_id"].unique())

8307

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [ ]:
_sum = df_receipt[["customer_id", "amount"]].groupby("customer_id").sum().query('not customer_id.str.startswith("Z")', engine = "python")
_mean = _sum.mean().values[0]
_sum.query('amount > @_mean').head(5)

,amount
customer_id,
CS001115000010,3044
CS001205000006,3337
CS001214000009,4685
CS001214000017,4132
CS001214000052,5639


---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [ ]:
pd.merge(df_customer, df_receipt[["customer_id", "amount"]].groupby("customer_id").sum(), how = "left", on = "customer_id").query('gender_cd == 1 & not customer_id.str.startswith("Z")', engine="python").head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,NaN
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,5088.0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,NaN
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,875.0
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,3122.0


---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [ ]:
from datetime import datetime

df_customer["application_date"] = df_customer["application_date"].apply(lambda x: datetime.strptime(str(x),"%Y%m%d"))
df_customer[["customer_id","application_date"]]

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
...,...,...
21966,CS002512000474,2017-11-10
21967,CS029414000065,2015-03-13
21968,CS012403000043,2015-04-06
21969,CS033512000184,2016-02-06


---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [ ]:
import math
_df = df_receipt[['customer_id', 'amount']].groupby('customer_id').sum().query('not customer_id.str.startswith("Z")',engine="python")
pd.concat([
    _df,
    _df["amount"].apply(lambda x: math.log10(x)).rename("log10")
], axis = 1).head(5)

,amount,log10
customer_id,,
CS001113000004,1298,3.113275
CS001114000005,626,2.796574
CS001115000010,3044,3.483445
CS001205000004,1988,3.298416
CS001205000006,3337,3.523356


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_product_1 = df_product.dropna()

In [ ]:
len(df_product), len(df_product_1)

(10030, 10023)

---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product_2 = df_product.fillna(df_product.mean().astype(int)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
